https://docs.ragas.io/en/stable/getstarted/evals/

https://docs.ragas.io/en/stable/getstarted/rag_eval/

## 📊 使用 Ragas 評估簡單的 LLM 應用程式

軟體開發流程，做 DevOps 的 CI/CD 迭代測試，來測試產出物的效能與品質。有完整的測試，才能為每個開發變動設立標準。

我們打造一個 RAG System，我們要如何知道這個 RAG System 的效能與品質呢？是看使用者的感覺決定 RAG 的好壞嗎？顯然不是。

### 📌 本指南目標

本指南的目的是展示如何使用 **Ragas** 測試與評估一個簡單的 LLM 應用工作流程。

✅ **Ragas** 是一個用於評估 Retrieval-Augmented Generation 系統表現的 Python 套件，能幫助你衡量回答的準確性、一致性與語意相關性。

✅ **Metrics 評估指標** 是用來量化評估 AI 應用效能的工具。透過這些指標，我們可以判斷整體應用程式以及其內部各個元件在測試資料上的表現好壞。指標提供數據基礎，協助進行比較、最佳化與決策，對於 AI 應用的開發與部署過程至關重要。（例如：faithfulness, answer_relevancy, context_precision 等）

✅ **Evaluation Data / Dataset** 是一組資料集，包含評估指標需要的各項參數，用來表示 Evaluation 的情境或是 test cases。將 RAG 的輸出加上資料集的參數輸入 Metrics 後，Metrics 回傳一個量化的結果，代表 RAG 的效能。

✅ **Evaluation 效能評估** 對於 RAG 系統，以相同的評估資料集做測試，獲得目前 RAG 的量化效能。並依據量化的結果，調整 RAG 系統的元件，例如 embedding, prompt, 使用的 model 等等，持續改善並持續評估。

---

### 🧪 評估場景：RAG

在本示範中，我們將評估一個 RAG System，
目標是確保模型產生的摘要**準確地涵蓋原始文本中的關鍵細節**

---

## 🛠️ 安裝 Ragas

若你想開始使用 **Ragas** 來對 RAG 系統進行評估，只需要使用 `pip` 指令來安裝：

In [2]:
!pip install ragas sacrebleu openai pandas tqdm tenacity

  Using cached ragas-0.2.15-py3-none-any.whl.metadata (9.0 kB)
  Using cached sacrebleu-2.5.1-py3-none-any.whl.metadata (51 kB)
  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached langchain-0.3.25-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_core-0.3.63-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_community-0.3.24-py3-none-any.whl.metadata (2.5 kB)
  Using cached langchain_openai-0.3.18-py3-none-any.whl.metadata (2.3 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached lxml-5.4.0-cp312-cp312-manylinux_2_28_aarch64.whl.metadata (3.5 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp312-cp312-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (12 kB)
  Usin

### 🔐 確保你已設定 OpenAI API 金鑰

請將你的 OpenAI API 金鑰設定為環境變數

In [21]:
import os

os.environ["AZURE_OPENAI_API_KEY"]=""
os.environ["AZURE_OPENAI_ENDPOINT"]=""
os.environ["OPENAI_API_VERSION"]="2024-12-01-preview"
os.environ["OPENAI_MODEL"]="gpt-4.1-mini"
#os.environ["OPENAI_MODEL"]="text-embedding-3-large"

if os.getenv("AZURE_OPENAI_API_KEY") is not None:
    print("AZURE_OPENAI_API_KEY is ready")
else:
    print("AZURE_OPENAI_API_KEY environment variable not found")

from openai import AzureOpenAI
openai_client = AzureOpenAI()

AZURE_OPENAI_API_KEY is ready


## 依照上一節的內容，將 RAG 系統開起來

In [8]:
import qdrant_client
from tenacity import retry, wait_random_exponential, stop_after_attempt

@retry(
    wait=wait_random_exponential(min=1, max=60),  # backoff 等待時間
    stop=stop_after_attempt(6),  # 最多重試 6 次
)
def get_embedding(text):
    res = openai_client.embeddings.create(
        model="text-embedding-3-large", input=[text]
    )
    return res.data[0].embedding

def search_docs(query, top_k=5):
    query_vect = get_embedding(query)
    hits = client.search(
        collection_name="Articles",
        query_vector=query_vect,
        limit=top_k,
        with_payload=True,
        using="title",
    )
    return [hit.payload["text"] for hit in hits]

from qdrant_client.models import NamedVector

def query_docs(query, top_k=5):
    query_vect = get_embedding(query)
    results = client.query_points(
        collection_name="Articles",
        query=query_vect,
        limit=5,
        with_payload=True,
        using="title"
    )

    payloads = [point.payload["answer"] for point in results.points]
    return payloads

@retry(
    wait=wait_random_exponential(min=1, max=60),  # backoff 等待時間
    stop=stop_after_attempt(6),  # 最多重試 6 次
)
def generate_answer(query, docs):
    context = "\n\n".join(docs)
    prompt = f"""根據以下內容回答問題：

1. 請用繁體中文回答
2. 依照內容產生回答
3. 附上內容原文作為依據，原文保留內容的原始語言
4.. 如果內容不包含就回答你不知道

內容：
{context}

問題：
{query}
"""

    res = openai_client.chat.completions.create(
        model="gpt-4.1-mini",  # 或你的 Azure 模型名稱
        messages=[
            {"role": "system", "content": "你是一個 helpful AI 助理"},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )
    return res.choices[0].message.content.strip()

client = qdrant_client.QdrantClient(
    host="qdrant",
    prefer_grpc=True,
)

### 🔄 RAG 查詢

In [9]:
query = "COVID 的全名是什麼"
#docs = search_docs(query)
docs = query_docs(query)
answer = generate_answer(query, docs)

print("\n🧠 回答：")
print(answer)


🧠 回答：
COVID 的全名是 "Coronavirus Disease 2019"（2019年冠狀病毒疾病）。

依據原文：
"WHO announced “COVID-19” as the name of this new disease on 11 February 2020, following guidelines previously developed with the World Organisation for Animal Health (OIE) and the Food and Agriculture Organization of the United Nations (FAO)."


### 📐 Ragas 提供的評估方法

Ragas 提供多種評估 LLM 應用效能的方法，我們稱這些為 **評估指標（metrics）**。

每個評估指標都需要一組預先定義的資料點（data points），
這些資料會用來計算「效能分數」，幫助你理解模型的表現。

---

### ✅ 使用非 LLM 評估指標（Non-LLM Metric）

以下是一個使用 **BLEU 分數**（`BleuScore`）的簡單評估範例，
這是一種不依賴大型語言模型的傳統語言相似度衡量方式，常用於摘要或翻譯任務。

https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/traditional/

In [10]:
from ragas import SingleTurnSample
from ragas.metrics import BleuScore

test_data = {
    "user_input": "summarise given text\nThe company reported an 8% rise in Q3 2024, driven by strong performance in the Asian market. Sales in this region have significantly contributed to the overall growth. Analysts attribute this success to strategic marketing and product localization. The positive trend in the Asian market is expected to continue into the next quarter.",
    "response": "The company experienced an 8% increase in Q3 2024, largely due to effective marketing strategies and product adaptation, with expectations of continued growth in the coming quarter.",
    "reference": "The company reported an 8% growth in Q3 2024, primarily driven by strong sales in the Asian market, attributed to strategic marketing and localized products, with continued growth anticipated in the next quarter."
}
metric = BleuScore()
test_data = SingleTurnSample(**test_data)
metric.single_turn_score(test_data)

0.13718598426177148

### 🔄 改用 RAG 的輸入與輸出

In [11]:
test_data = {
    "user_input": "What is the full name of COVID?",
    "response": 'The full name of COVID is "Coronavirus Disease 2019"',
    "reference": "WHO announced “COVID-19” as the name of this new disease on 11 February 2020, following guidelines previously developed with the World Organisation for Animal Health (OIE) and the Food Agriculture Organization of the United Nations (FAO)."
}
metric = BleuScore()
test_data = SingleTurnSample(**test_data)
metric.single_turn_score(test_data)

0.004475925962507959

## 📋 我們在這裡使用了：

* 一筆測試資料（**test sample**），包含以下欄位：

  * `user_input`：使用者輸入
  * `response`：由 LLM 模型生成的輸出
  * `reference`：理想或預期的輸出結果（作為評估依據）

* 一個非 LLM 型的評估指標：`BleuScore`

---

## ⚠️ 此方法的兩大限制：

### 1️⃣ 需要大量準備工作

評估這類應用時，**必須為每個輸入預先準備對應的理想輸出（reference）**。
這種方式不僅耗時，還很難在大規模應用中維護。

### 2️⃣ 評分結果可能不準確

即使實際輸出（`response`）與預期輸出（`reference`）非常相似，
**`BleuScore` 所計算出的分數仍可能偏低**。

這是傳統非語言模型指標（例如 `Bleu`, `ROUGE`, `METEOR` 等）的既有問題：
它們**只依據詞彙重合率進行評分**，而無法理解語意上的相似性。

---

### ✅ 建議使用

為了解決上述問題，建議使用 **Ragas 提供的 LLM 驅動評估指標**（例如：`faithfulness`, `answer_relevancy`, `context_precision` 等），
這些指標會使用語言模型來比較輸出與參考答案的語意一致性，更能反映實際表現。

## 🤖 使用 LLM 為基礎的評估指標進行評估

相較於傳統的非 LLM 指標（如 BLEU 或 ROUGE），**LLM-based metrics** 能理解語意與上下文，
可更準確地評估大型語言模型（LLM）所生成的回應品質。

LLM-based 指標（例如 `faithfulness`, `answer_relevancy`, `context_precision` 等）能夠深入理解語意、上下文與問題間的關係，提供更準確的評估結果，適合用於 RAG 系統。

---

## 載入資料集

### 📁 資料集簡介：`covidqa`

這是一組基於 **COVID-19 文獻資料** 建立的問答資料集，目的是幫助開發者訓練自然語言理解（NLU）與問答系統，以回應與疫情相關的問題。

---

### 📄 檔案：`community.csv` 是什麼？

這個 `community.csv` 是該資料集中的一個檔案，根據名稱與用途，它大致包含從 COVID-19 社群討論（例如研究社群、論壇、問答平台等）中擷取出來的 **問答對（Question-Answer Pairs）**。

---

### 🔍 用途

* 用於訓練問答模型（如：BERT、GPT + 向量資料庫）。
* 適合建立 Retrieval-Augmented Generation (RAG) 系統。
* 可搭配向量嵌入技術（如 OpenAI Embedding + Qdrant）建立問答應用。

# 收集評估資料（Evaluation Data）

為了收集評估資料，我們首先需要準備一組查詢（queries），用來對 RAG 系統進行測試。

## 步驟說明

1. **準備查詢集**  
   準備一組查詢，這些查詢將被送入 RAG 系統。

2. **執行查詢**  
   將這些查詢透過 RAG 系統執行，並收集以下資料：
   - 系統回應（response）
   - 為每個查詢所擷取的上下文內容（retrieved_contexts）

3. **（選用）準備標準答案**  
   可以選擇為每個查詢準備一組黃金標準答案（golden answers），用來進一步評估系統的表現。

---

### 🔍 原始資料集取樣，產生 Evaluation 資料集

* 使用更完整的資料及進行測試會花更多時間
* 也會花更多成本在 OpenAI API

為了節省時間，這邊只採用原始資料集中的 30 個 sample

---

### 🔍 產生 Evaluation 資料集

* 請 COVID 專家，產生專業的 COVID 問答，作為資料集
* 使用 LLM + testset generator 產生全新的 COVID 問答資料集。由於 LLM 普及，成本變得很低
* 在剛開始做 RAG 前，保留一部分原始資料集，不要寫入 Qdrant，作為 Evaluation 資料集


In [14]:
import pandas as pd

article_df = pd.read_csv('/home/jovyan/community.csv')
eval_samples=article_df.sample(n=10)
eval_samples

,question_id,title,question,answer_id,answer,answer_type,wrong_answer,wrong_answer_type,url,source
502,52306,Has the WHO said anything about travel restric...,The WHO was against travel restrictions as the...,52307,The WHO released a joined statement with the I...,Reasonable,Donald Trump tweeted the following on August 2...,Random,politics.stackexchange.com,general
189,11353,Why does the FASTA sequence for coronavirus lo...,I'm looking at a genome sequence for 2019-nCoV...,11358,That is the correct sequence for 2019-nCov. Co...,Accepted,I just searched GenBank accepts nucleotide seq...,Bad Answers,bioinformatics.stackexchange.com,biomedical
489,51091,Does the FDA regulate off-label use of pharmac...,In today's press conference (3/19/2020) the Pr...,51093,\n Considering today's assertion that FDA has...,Reasonable,"Yes, as has been referenced many times, Congre...",Random,politics.stackexchange.com,general
300,145539,How does the COVID-19 crisis affect durations ...,One question we often get on this site in one ...,145542,"I am an associate editor for two journals, and...",Reasonable,\n How should I modify these in lights of the...,Bad Answers,academia.stackexchange.com,expert
475,52610,What did the US CDC say about COVID-19 human-t...,"From (answers to) a related question, the WHO ...",52613,"On January 17th, the CDC announced that COVID-...",Accepted,A conservative minority government would form....,Random,politics.stackexchange.com,general
387,121808,"Why don't companies, whose share price plunged...",COVID-19 has caused many large-caps' stocks to...,121813,\n Why don't they reverse stock split to upli...,Reasonable,Maybe you are thinking of a buy-back?\n\nIt is...,Bad Answers,money.stackexchange.com,general
259,459046,Parameter Estimation for the SIRD model via Ka...,0 Introduction\n\nThis is my third attempt in ...,459081,Nice write-up. I think eq#23 is not quite corr...,Accepted,Here's an image where different R2 Scores are ...,Random,stats.stackexchange.com,expert
255,455202,Is the COVID-19 pandemic curve a Gaussian curve?,"We've all heard a lot about ""flattening the cu...",455639,It seems like there are three questions here:\...,Accepted,"In fact, from the underlying processes that ge...",Bad Answers,stats.stackexchange.com,expert
519,52643,How does libertarianism handle contagious dise...,"In libertarianism, each person is responsible ...",52654,While I haven't found a reference discussing t...,Reasonable,"\nFirst of all, IMHO this question can't be an...",Random,politics.stackexchange.com,general
390,122035,Why are my country's banks buying their own sh...,"In light of the Covid-19 crisis, my country's ...",122041,Assume there is a corporation that makes spoon...,Reasonable,Because they have money and stocks are cheap. ...,Bad Answers,money.stackexchange.com,general


In [15]:
from tqdm import tqdm

dataset = []

for index, row in tqdm(eval_samples.iterrows(), total=len(eval_samples), desc="Building eval dataset"):
    query = row["title"]
    relevant_docs = query_docs(query)
    response = generate_answer(query, relevant_docs)
    dataset.append(
        {
            "user_input": query,
            "retrieved_contexts": relevant_docs,
            "response": response,
            "reference": row["answer"]
        }
    )

Building eval dataset: 100%|██████████| 10/10 [01:08<00:00,  6.85s/it]


### 檢視產生的評估資料集

In [16]:
dataset[0]

{'user_input': 'Has the WHO said anything about travel restrictions since the end of February?',
 'retrieved_contexts': ['The WHO released a joined statement with the ICAO ([on the WHO website it\'s dated 11 March 2020). It does not mention travel restrictions directly. It does, however, stress cooperation at different levels, for example between "aviation and health authorities". \n\nMy interpretation is that the WHO is leaving such restrictions up to the states, at least in their public statements. \n',
  'Probably because prior WHO research was rather skeptical of their ultimate effectiveness:  \n\n\n  Screening and quarantining entering travelers at international borders did not substantially delay virus introduction in past pandemics, except in some island countries, and will likely be even less effective in the modern era.\n\n\nThat paper was co-authored (on behalf of the WHO) by employees of the [US] CDC (lead author actually), the ECDC, the UK\'s HPA, Australian DHA, and some o

In [17]:
from ragas import EvaluationDataset
evaluation_dataset = EvaluationDataset.from_list(dataset)

# 評估（Evaluate）

我們已成功收集評估資料，接下來可以使用一組常見的 RAG 評估指標（evaluation metrics）對我們的 RAG 系統進行評估。

## 評估步驟

1. **使用收集到的資料集**  
   基於先前收集的查詢、系統回應及擷取的上下文，進行模型效能評估。

2. **選擇評估指標**  
   使用常見的 RAG 評估指標，例如：
   - 回答正確性（Answer Correctness）
   - 上下文相關性（Context Relevance）
   - 回應完整性（Answer Completeness）

3. **選擇評估模型**  
   你可以選擇任意一個 LLM 作為評估器（Evaluator LLM）來執行這些評估任務。

https://docs.ragas.io/en/v0.1.21/howtos/customisations/azure-openai.html

In [18]:
from langchain_openai.chat_models import AzureChatOpenAI
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from ragas import evaluate

azure_model = AzureChatOpenAI(
    model="gpt-4.1-mini",
    validate_base_url=False,
)

# init the embeddings for answer_relevancy, answer_correctness and answer_similarity
azure_embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
)

https://docs.ragas.io/en/stable/concepts/metrics/overview/#different-types-of-metrics

# 評估（Evaluate）

在成功收集完評估資料後，我們就可以使用一組常見的 RAG 評估指標，針對收集到的資料集來評估 RAG 系統的效能。

你可以選擇任何一個模型作為評估用的 LLM（Evaluator LLM）來進行評分與比較。

---

# 評估指標概觀（Overview of Metrics）

「指標（Metric）」是用來量化評估 AI 應用效能的工具。透過這些指標，我們可以判斷整體應用程式以及其內部各個元件在測試資料上的表現好壞。指標提供數據基礎，協助進行比較、最佳化與決策，對於 AI 應用的開發與部署過程至關重要。

## 指標的重要性

- **元件選擇（Component Selection）**  
  可使用指標來比較 AI 應用中的不同元件（如 LLM、Retriever、Agent 組態等）在自有資料上的表現，從多個選項中選出最佳組合。

- **錯誤診斷與除錯（Error Diagnosis and Debugging）**  
  指標能協助找出應用中表現不佳或發生錯誤的元件，有助於釐清問題並加以改進。

- **持續監控與維運（Continuous Monitoring and Maintenance）**  
  指標可用於長期追蹤 AI 應用的表現，幫助偵測如資料漂移、模型效能下降或用戶需求變化等問題，並及時調整應對。

---

# 常見 RAG 評估指標說明

## 1. LLM Context Recall（LLM 上下文召回率）
- **說明**：衡量 LLM 回應中是否使用了檢索到的上下文資訊。
- **目標**：確保模型的回答有依據地參考了相關的背景資料。

## 2. Faithfulness（忠實度）
- **說明**：評估回應是否忠實反映檢索到的內容。
- **目標**：避免 LLM 編造（hallucinate）資訊，確保回答與來源資料一致。

## 3. Factual Correctness（事實正確性）
- **說明**：判斷 LLM 回應的事實是否正確，不僅限於上下文來源，也包括一般常識與外部知識。
- **目標**：減少錯誤或誤導性的回答。

## 4. Noise Sensitivity（雜訊敏感度）
- **說明**：評估模型在面對含有雜訊或不相關資訊的上下文時，是否仍能做出正確判斷。
- **目標**：強化模型的抗雜訊能力，提升實用性。

## 5. Response Relevancy（回應相關性）
- **說明**：評估 LLM 回應是否與使用者查詢高度相關。
- **目標**：確保回答內容精準聚焦於問題本身。

## 6. Context Entity Recall（上下文實體召回率）
- **說明**：檢查模型是否成功提取上下文中與查詢相關的重要實體（例如人名、地名、產品名等）。
- **目標**：提升資訊擷取的完整性與語意理解。

---

這些指標可以單獨或組合使用，幫助開發者全面了解 RAG 系統在真實應用中的表現，進而針對特定弱點進行優化。

In [19]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, NoiseSensitivity, ResponseRelevancy, ContextEntityRecall

result = evaluate(
    evaluation_dataset, 
    metrics=[
        LLMContextRecall(), 
        Faithfulness(), 
        FactualCorrectness(),
        #NoiseSensitivity(),
        #ResponseRelevancy(),
        #ContextEntityRecall()
    ],
    llm=azure_model, 
    embeddings=azure_embeddings
)

result

Evaluating: 100%|██████████| 30/30 [02:10<00:00,  4.35s/it]


{'context_recall': 0.9673, 'faithfulness': 0.9728, 'factual_correctness(mode=f1)': 0.7100}

In [20]:
df = result.to_pandas()
df.head()

,user_input,retrieved_contexts,response,reference,context_recall,faithfulness,factual_correctness(mode=f1)
0,Has the WHO said anything about travel restric...,[The WHO released a joined statement with the ...,根據內容，世界衛生組織（WHO）在二月底之後，沒有發布更多關於旅行限制的最新新聞稿，尤其是在...,The WHO released a joined statement with the I...,0.75,0.833333,0.18
1,Why does the FASTA sequence for coronavirus lo...,[That is the correct sequence for 2019-nCov. C...,根據內容，冠狀病毒的FASTA序列看起來像DNA而非RNA的原因是：\n\n冠狀病毒是RNA...,That is the correct sequence for 2019-nCov. Co...,1.00,1.000000,0.97
2,Does the FDA regulate off-label use of pharmac...,[\n Considering today's assertion that FDA ha...,根據內容，FDA 並不直接管制藥物的「off-label」（非核准適應症）使用。內容中提到：...,\n Considering today's assertion that FDA has...,1.00,1.000000,0.45
3,How does the COVID-19 crisis affect durations ...,"[I am an associate editor for two journals, an...",根據內容，COVID-19 疫情對於審稿和編輯處理時間的影響如下：\n\n1. 大部分工作都...,"I am an associate editor for two journals, and...",1.00,1.000000,0.89
4,What did the US CDC say about COVID-19 human-t...,"[On January 17th, the CDC announced that COVID...",根據內容，2020年1月14日至21日期間，美國CDC對COVID-19（當時稱為2019-...,"On January 17th, the CDC announced that COVID-...",1.00,1.000000,0.73
